### In this notebook, we will apply an exploratory data analysis (EDA) pipeline to:

- Load the train CSV file into Pandas dataframes for exploration.
- Use `Pandas Profiling` to profile the training dataframe by generating a data analysis report.
- Examine the generated data analysis report.
- Recommend any transformations to be applied on the data based on the analysis.
- Implement the transformations logic.
- Apply the transformations logic on the three data splits (train, validate, test).
- Save the updated versions of the datasets as CSV files. 

In [1]:
# Import the required libraries
import numpy as np
import pandas as pd
from ydata_profiling import ProfileReport
from sklearn.preprocessing import StandardScaler
from IPython.display import display

# Data source & destination path
in_out = '../data/'

In [2]:
# Load the train CSV file into Pandas dataframe for analysis
train_df = pd.read_csv(f'{in_out}train.csv')

# Generate a standard profiling report
profile = ProfileReport(train_df, title="Training Set Profiling Report")

# Export the report to HTML & JSON files
profile.to_file("train_profile_report.html")
profile.to_file("train_profile_report.json")

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

Render JSON:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
# OPTIONAL: Convert the HTML report to PDF report using Pandoc as follows
# Open Windows Powershell as Administrator and run the following commands:
# choco install pandoc
# choco install rsvg-convert
# choco install miktex
# pandoc -s -r html .\train_profile_report.html --pdf-engine=xelatex -o train_profile_report.pdf

### Profiling Report Findings and Recommendations

Here are the key findings and recommendations from my analysis of the [training data profiling report](train_profile_report.html):

**Missing Values:**

No columns have missing values, so no imputation is needed.

**Duplicate Rows:**

There are 410 duplicate rows (10.5% of the data). 

I would recommend removing the duplicates to prevent overfitting models.

**Feature Distributions:**

The following features are right skewed based on their positive skewness values in the report:

- fixed acidity (skewness = 1.75)
- volatile acidity (skewness = 1.57)
- chlorides (skewness = 5.67)
- free sulfur dioxide (skewness = 0.89)
- sulphates (skewness = 1.81)
- alcohol (skewness = 0.57)

I would apply a log transformation to each of these features to make their distributions more normal before model building.

**Outliers:**

No major outliers identified in the analysis. 

However, some models like linear regression are sensitive to outliers. 

I suggest winsorizing / clipping extreme values as a precaution.

**Feature Scaling:**

The features are in different scales based on their min and max values. 

To standardize them, I would apply `StandardScaler()` from `sklearn.preprocessing` to transform each feature to have mean 0 and standard deviation 1.

This scaling would help optimization converge faster during model training.

**Highly Correlated Features:**

Based on the correlation matrix, the following numerical feature pairs are highly correlated:

- residual sugar & density (correlation = 0.52)
- chlorides & density (correlation = 0.59)
- free sulfur dioxide & total sulfur dioxide (correlation = 0.74)

For each correlated pair, I would create a new feature by taking the mean or ratio of the two features. This results in the following new features:

- residual_sugar_density_mean
- chlorides_density_ratio
- sulfur_dioxide_mean

Then, I will remove the correlated features from the dataset. 

In [2]:
def remove_duplicates(df):
    df.drop_duplicates(inplace=True)
    return df
    
def apply_log_transformation(df, features):
    for feature in features:
        df[feature] = df[feature].apply(lambda x: np.log1p(x))
    return df

def detect_outliers(df, features):
    outlier_info = {}
    for feature in features:
        # Calculate the 95th percentile for the feature
        percentile_95 = df[feature].quantile(0.95)
        
        # Find the indices where the feature values exceed the 95th percentile
        outlier_indices = df[df[feature] > percentile_95].index

        if len(outlier_indices) > 0:
            # Store the lower and upper limit of outliers for the feature
            lower_limit = df[feature].min()
            upper_limit = percentile_95
            
            # Update the outlier information dictionary
            outlier_info[feature] = {'lower_limit': lower_limit, 'upper_limit': upper_limit, 'outlier_indices': outlier_indices}
    return outlier_info
    
def winsorize_outliers(df, features, lower_limits, upper_limits):
    for feature, lower_limit, upper_limit in zip(features, lower_limits, upper_limits):
        df[feature] = df[feature].clip(lower=lower_limit, upper=upper_limit)
    return df

def apply_feature_scaling(df, exclude_cols):
    scaler = StandardScaler()
    df_scaled = df.copy()
    for col in df.columns:
        if col not in exclude_cols:
            df_scaled[col] = scaler.fit_transform(df[[col]])
    return df_scaled

def create_new_features(df):
    df['residual_sugar_density_mean'] = (df['residual sugar'] + df['density']) / 2
    df['chlorides_density_ratio'] = df['chlorides'] / df['density']
    df['sulfur_dioxide_mean'] = (df['free sulfur dioxide'] + df['total sulfur dioxide']) / 2
    df.drop(['residual sugar', 'chlorides', 'density', 'free sulfur dioxide', 'total sulfur dioxide'], axis=1, inplace=True)
    return df

In [3]:
# Load the CSV files into Pandas dataframes for the data transformations
train_df = pd.read_csv(f'{in_out}train.csv')
val_df = pd.read_csv(f'{in_out}validate.csv')
test_df = pd.read_csv(f'{in_out}test.csv')

# Transform train_df as required
train_df = remove_duplicates(train_df)
features = ['fixed acidity', 'volatile acidity', 'chlorides', 'free sulfur dioxide', 'sulphates', 'alcohol']
train_df = apply_log_transformation(train_df, features)
outlier_info = detect_outliers(train_df, features)
features = []
lower_limits = []
upper_limits = []
for feature, info in outlier_info.items():
    lower_limit = info['lower_limit']
    upper_limit = info['upper_limit']
    outlier_indices = info['outlier_indices']
    outlier_count = len(outlier_indices)
    print(f"Feature: {feature}")
    print(f"Lower Limit: {lower_limit}")
    print(f"Upper Limit: {upper_limit}")
    print(f"Outlier count: {outlier_count}")
    print()
    features.append(feature)
    lower_limits.append(lower_limit)
    upper_limits.append(upper_limit)
train_df = winsorize_outliers(train_df, features, lower_limits, upper_limits)
train_df = apply_feature_scaling(train_df, ['wine_type', 'quality'])
train_df = create_new_features(train_df)
display(train_df.head(5))

Feature: fixed acidity
Lower Limit: 1.5686159179138452
Upper Limit: 2.388762789235098
Outlier count: 161

Feature: volatile acidity
Lower Limit: 0.07696104113612832
Upper Limit: 0.5187937934151676
Outlier count: 166

Feature: chlorides
Lower Limit: 0.008959741371471904
Upper Limit: 0.09893994785490356
Outlier count: 164

Feature: free sulfur dioxide
Lower Limit: 1.0986122886681096
Upper Limit: 4.127134385045092
Outlier count: 165

Feature: sulphates
Lower Limit: 0.1988508587451652
Upper Limit: 0.5933268452777344
Outlier count: 165

Feature: alcohol
Lower Limit: 2.1972245773362196
Upper Limit: 2.617395832834079
Outlier count: 164



,fixed acidity,volatile acidity,citric acid,pH,sulphates,alcohol,quality,wine_type,residual_sugar_density_mean,chlorides_density_ratio,sulfur_dioxide_mean
0,1.943612,0.609132,0.611565,-1.011681,-0.021103,0.477217,6.0,white,-0.230202,-5.309967,0.184676
1,0.455066,2.106564,-1.496435,1.045176,-0.101079,-1.071319,5.0,red,-0.071295,0.899096,-0.451677
2,1.197274,-0.367374,0.203565,0.484215,1.616734,1.131435,7.0,red,-0.273300,1.665372,-1.864449
3,-0.974228,-0.974543,0.543565,0.234899,1.476658,1.051930,5.0,white,-0.908618,1.102921,0.400251
4,-0.665612,-0.367374,0.815565,-0.450720,-0.344227,-1.170216,5.0,white,0.337231,-1.053460,1.077442


In [4]:
# Transform val_df as required
val_df = remove_duplicates(val_df)
features = ['fixed acidity', 'volatile acidity', 'chlorides', 'free sulfur dioxide', 'sulphates', 'alcohol']
val_df = apply_log_transformation(val_df, features)
outlier_info = detect_outliers(val_df, features)
features = []
lower_limits = []
upper_limits = []
for feature, info in outlier_info.items():
    lower_limit = info['lower_limit']
    upper_limit = info['upper_limit']
    outlier_indices = info['outlier_indices']
    outlier_count = len(outlier_indices)
    print(f"Feature: {feature}")
    print(f"Lower Limit: {lower_limit}")
    print(f"Upper Limit: {upper_limit}")
    print(f"Outlier count: {outlier_count}")
    print()
    features.append(feature)
    lower_limits.append(lower_limit)
    upper_limits.append(upper_limit)
val_df = winsorize_outliers(val_df, features, lower_limits, upper_limits)
val_df = apply_feature_scaling(val_df, ['wine_type', 'quality'])
val_df = create_new_features(val_df)
display(val_df.head(5))

Feature: fixed acidity
Lower Limit: 1.589235205116581
Upper Limit: 2.3608540011180215
Outlier count: 62

Feature: volatile acidity
Lower Limit: 0.07696104113612832
Upper Limit: 0.5187937934151676
Outlier count: 62

Feature: chlorides
Lower Limit: 0.011928570865273802
Upper Limit: 0.1032331783574455
Outlier count: 63

Feature: free sulfur dioxide
Lower Limit: 0.6931471805599453
Upper Limit: 4.127134385045092
Outlier count: 61

Feature: sulphates
Lower Limit: 0.23111172096338664
Upper Limit: 0.5822156198526637
Outlier count: 58

Feature: alcohol
Lower Limit: 2.2407096892759584
Upper Limit: 2.617395832834079
Outlier count: 61



,fixed acidity,volatile acidity,citric acid,pH,sulphates,alcohol,quality,wine_type,residual_sugar_density_mean,chlorides_density_ratio,sulfur_dioxide_mean
0,2.190564,0.046895,2.155839,0.232762,1.633165,0.637971,7.0,red,0.539197,0.852864,-1.983157
1,0.033769,-0.024094,0.569488,1.245718,0.615143,-0.055895,5.0,white,-0.445063,9.878947,-0.759966
2,-0.477113,0.256704,1.052290,-0.843503,0.307875,-0.328727,6.0,white,-0.717048,-4.006448,0.115675
3,0.698372,1.115392,1.190234,-0.083786,-1.627970,-0.421313,5.0,white,0.242498,-2.960732,0.423347
4,-0.166695,-1.078671,0.293601,0.296072,1.976996,-0.897240,7.0,white,-0.623175,1.976402,0.380723


In [5]:
# Transform test_df as required
test_df = remove_duplicates(test_df)
features = ['fixed acidity', 'volatile acidity', 'chlorides', 'free sulfur dioxide', 'sulphates', 'alcohol']
test_df = apply_log_transformation(test_df, features)
outlier_info = detect_outliers(test_df, features)
features = []
lower_limits = []
upper_limits = []
for feature, info in outlier_info.items():
    lower_limit = info['lower_limit']
    upper_limit = info['upper_limit']
    outlier_indices = info['outlier_indices']
    outlier_count = len(outlier_indices)
    print(f"Feature: {feature}")
    print(f"Lower Limit: {lower_limit}")
    print(f"Upper Limit: {upper_limit}")
    print(f"Outlier count: {outlier_count}")
    print()
    features.append(feature)
    lower_limits.append(lower_limit)
    upper_limits.append(upper_limit)
test_df = winsorize_outliers(test_df, features, lower_limits, upper_limits)
test_df = apply_feature_scaling(test_df, ['wine_type', 'quality'])
test_df = create_new_features(test_df)
display(test_df.head(5))

Feature: fixed acidity
Lower Limit: 1.6486586255873816
Upper Limit: 2.3565885663384765
Outlier count: 63

Feature: volatile acidity
Lower Limit: 0.07696104113612832
Upper Limit: 0.5128236264286637
Outlier count: 59

Feature: chlorides
Lower Limit: 0.011928570865273802
Upper Limit: 0.09216878454782193
Outlier count: 63

Feature: free sulfur dioxide
Lower Limit: 0.6931471805599453
Upper Limit: 4.0943445622221
Outlier count: 62

Feature: sulphates
Lower Limit: 0.22314355131420976
Upper Limit: 0.5766133643039938
Outlier count: 58

Feature: alcohol
Lower Limit: 2.2407096892759584
Upper Limit: 2.6100697927420065
Outlier count: 60



,fixed acidity,volatile acidity,citric acid,pH,sulphates,alcohol,quality,wine_type,residual_sugar_density_mean,chlorides_density_ratio,sulfur_dioxide_mean
0,-1.120154,-0.317796,0.224866,-0.374328,1.369292,0.157738,5.0,white,-0.595651,1.699143,-0.304219
1,-0.367534,-0.465181,1.191257,-0.189164,-0.314037,-0.869395,5.0,white,0.450530,-0.022487,1.208948
2,-0.575350,-0.844129,0.362922,0.674936,1.811890,0.845291,7.0,white,-0.805796,0.208846,0.845219
3,-0.681345,0.587470,-0.534442,1.353873,0.107481,-0.578967,6.0,white,1.523108,-0.169881,-0.063497
4,-0.065720,0.653610,-0.879582,1.045265,0.353829,0.420973,5.0,red,-0.303607,18.960999,-0.961104


In [6]:
# Persist the data splits as CSV files into the file system
train_df.to_csv(f'{in_out}train_transformed.csv', index=False)
val_df.to_csv(f'{in_out}validate_transformed.csv', index=False)
test_df.to_csv(f'{in_out}test_transformed.csv', index=False)

### Profile the transformed training dataframe and examine the data analysis report for verification

In [7]:
# Load the train CSV file into Pandas dataframe for analysis
train_df = pd.read_csv(f'{in_out}train_transformed.csv')

# Generate a standard profiling report
profile = ProfileReport(train_df, title="Transformed Training Set Profiling Report")

# Export the report to HTML & JSON files
profile.to_file("transformed_train_profile_report.html")
profile.to_file("transformed_train_profile_report.json")

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

Render JSON:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]